In [105]:
# Dependancies
import pandas as pd, requests, json
import os
import csv

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base

# Define Declarative Base 
Base = declarative_base()

# Library OpenCageGeocode => to get lats and longs for each country
from opencage.geocoder import OpenCageGeocode

In [39]:
# Creating an engine, sqlite database and connection to the engine
engine = create_engine("sqlite:///db/wine.sqlite", echo=False)
conn = engine.connect()

In [45]:
# Use this to delete tables
# engine.execute('DROP TABLE map_wine_table')

# Clear the metadata object, too
# Base.metadata.clear()

# Inspecting the Table to make sure the table is there
# inspector = inspect(engine)
# inspector.get_table_names()

['wine_table']


# Exploratory Analysis of the Data

In [ ]:
# Read in CSV File
raw_df = pd.read_csv('db/wine_library.csv', dtype = {'Zip': 'str'})
raw_df.head(10)

In [107]:
# Inspecting the data 
# NOTE: The lowest point is 80, the highest is 100, so have to put the ratings in the range for the ml label later on
raw_df.describe()

,Unnamed: 0,points,price
count,150930.000000,150930.000000,137235.000000
mean,75464.500000,87.888418,33.131482
std,43569.882402,3.222392,36.322536
min,0.000000,80.000000,4.000000
25%,37732.250000,86.000000,16.000000
50%,75464.500000,88.000000,24.000000
75%,113196.750000,90.000000,40.000000
max,150929.000000,100.000000,2300.000000


In [54]:
# Shape of the data
raw_df.shape

(150930, 11)

In [55]:
# See if there are any NaN values
raw_df.isnull().sum()

Unnamed: 0         0
country            5
description        0
designation    45735
points             0
price          13695
province           5
region_1       25060
region_2       89977
variety            0
winery             0
dtype: int64

In [108]:
# Drop designation & region_2 columns (don't need those)
dropped_df = raw_df.drop(['designation', 'Unnamed: 0'], axis = 1) 
dropped_df.head(2)

,country,description,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez


In [109]:
# Drop all NaN values in region_1 and price columns
# NOTE: if there is a missing values in region 1 then the data was entered incorrectly
# NOTE: Only dropping NaN values from country and price. If I dropped NaN values from region_2 then I only have 9 
# countries, losing too much data.
new_df = dropped_df.dropna(subset=['price', 'country'])

In [110]:
# The shape of a clean dataset 
new_df.shape

(137230, 9)

In [147]:
new_df.head(100)

,country,description,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Mac Watson honors the memory of a wine once ma...,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La Bégude, named aft...",95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude
5,Spain,"Deep, dense and pure from the opening bell, th...",95,73.0,Northern Spain,Toro,NaN,Tinta de Toro,Numanthia
6,Spain,Slightly gritty black-fruit aromas include a s...,95,65.0,Northern Spain,Toro,NaN,Tinta de Toro,Maurodos
7,Spain,Lush cedary black-fruit aromas are luxe and of...,95,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
8,US,This re-named vineyard was formerly bottled as...,95,65.0,Oregon,Chehalem Mountains,Willamette Valley,Pinot Noir,Bergström
9,US,The producer sources from two blocks of the vi...,95,60.0,California,Sonoma Coast,Sonoma,Pinot Noir,Blue Farm


# Load the Data into sqlite database

In [18]:
# Creating a Class & table 
class Wine(Base):
    __tablename__ = 'wine_table'
    id = Column(Integer, primary_key=True)
    country = Column(String(255))
    description = Column(String(1000))
    points = Column(Float)
    price = Column(Float)
    province = Column(String(255))
    region_1 = Column(String(255))
    region_2 = Column(String(255))
    variety = Column(String(255))
    winery = Column(String(255))

In [19]:
Base.metadata.create_all(conn)

In [20]:
# Transfer the data to sqlite table
new_df.to_sql(name='wine_table', con=conn, if_exists='append', index=False)

In [21]:
# Inspecting the Table to make sure the table is there
inspector = inspect(engine)
inspector.get_table_names()

['wine_table']

In [22]:
# Checking the data 
engine.execute('SELECT * FROM wine_table LIMIT 20').fetchall()

[(1, 'US', 'This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel  ... (57 characters truncated) ... ubtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy 2022–2030.', 96.0, 235.0, 'California', 'Napa Valley', 'Napa', 'Cabernet Sauvignon', 'Heitz'),
 (2, 'Spain', 'Ripe aromas of fig, blackberry and cassis are softened and sweetened by a slathering of oaky chocolate and vanilla. This is full, layered, intense an ... (20 characters truncated) ... alate, with rich flavors of chocolaty black fruits and baking spices. A toasty, everlasting finish is heady but ideally balanced. Drink through 2023.', 96.0, 110.0, 'Northern Spain', 'Toro', None, 'Tinta de Toro', 'Bodega Carmen Rodríguez'),
 (3, 'US', 'Mac Watson honors the memory of a wine once made by his mother in this tremendously delicious, balanced and complex b

# Data Engineering for Leaflet part

In [146]:
new_df["winery"].unique()
winery_data = new_df.groupby('winery').agg({'points':'mean', 'price':'mean'})

In [144]:
# Find best wineries
winery_data.sort_values(by=['points'], ascending = False)

,points,price
winery,,
Sloan,100.000000,245.000000
Mascarello Giuseppe e Figlio,99.000000,175.000000
Clos de Tart,98.000000,319.000000
Domaine Bruno Clair,97.000000,281.000000
Au Sommet,97.000000,250.000000
Gandona,97.000000,190.000000
Vieux Château Certan,97.000000,130.000000
Cardinale,97.000000,215.000000
Ovid,97.000000,195.000000


In [112]:
# Check for the unique countries 
new_df["country"].unique()

array(['US', 'Spain', 'France', 'Italy', 'New Zealand', 'Bulgaria',
       'Argentina', 'Australia', 'Portugal', 'Israel', 'South Africa',
       'Greece', 'Chile', 'Morocco', 'Romania', 'Germany', 'Canada',
       'Moldova', 'Hungary', 'Austria', 'Croatia', 'Slovenia', 'India',
       'Turkey', 'Macedonia', 'Lebanon', 'Serbia', 'Uruguay',
       'Switzerland', 'Albania', 'Bosnia and Herzegovina', 'Brazil',
       'Cyprus', 'Lithuania', 'Japan', 'China', 'South Korea', 'Ukraine',
       'England', 'Mexico', 'Georgia', 'Montenegro', 'Luxembourg',
       'Slovakia', 'Czech Republic', 'US-France'], dtype=object)

In [113]:
# Get the averages for each country 
map_data = new_df.groupby('country').agg({'points':'mean', 'price':'mean'})
map_data.head(2)

,points,price
country,,
Albania,88.000000,20.000000
Argentina,86.001074,20.794881


In [114]:
# Reset the index in order to convert df into a list of dictionaries
map_df = map_data.reset_index()
map_df.head(2)

,country,points,price
0,Albania,88.000000,20.000000
1,Argentina,86.001074,20.794881


In [115]:
# OpenCageGeocode
# Save API key in the variable 
key = "735fa8c57707473ab1e9c3742a4e9e2c"

In [116]:
# Assig the key 
geocoder = OpenCageGeocode(key)

In [117]:
# Create empty lists to save the results
list_lat = []   
list_long = []

# Iterate over rows in dataframe
for index, row in map_df.iterrows(): 

    Country = row['country']    
    query = str(Country)

    results = geocoder.geocode(query)   
    lat = results[0]['geometry']['lat']
    long = results[0]['geometry']['lng']

    list_lat.append(lat)
    list_long.append(long)

# Create new columns from lists    
map_df['lat'] = list_lat   
map_df['lon'] = list_long

map_df.head(2)

,country,points,price,lat,lon
0,Albania,88.000000,20.000000,41.000028,19.999962
1,Argentina,86.001074,20.794881,-34.996496,-64.967282


In [140]:
# Find the top 5 countries for qualities for the map visualization
map_df.sort_values(by=['points'], ascending = False)

,country,points,price,lat,lon
13,England,92.750000,47.500000,52.795479,-0.540240
3,Austria,89.190898,31.192106,47.200034,13.199959
16,Germany,88.610567,39.011078,51.083420,10.423447
14,France,88.603314,45.619885,46.603354,1.888334
21,Italy,88.445006,37.547913,42.638426,12.674297
7,Canada,88.221649,34.628866,61.066692,-107.991707
32,Portugal,88.176724,26.332615,40.033263,-7.889626
36,Slovenia,88.172840,28.061728,45.813311,14.480837
30,Morocco,88.166667,18.833333,31.172821,-7.336248
41,Turkey,88.140000,25.800000,38.959759,34.924965


In [125]:
# drop any rows that lack lat/long data if any
df_geo = map_df.dropna(subset=['lat', 'lon'], axis=0, inplace=False)

print('We have {} geotagged rows'.format(len(df_geo)))
df_geo.tail()

We have 46 geotagged rows


,country,points,price,lat,lon
41,Turkey,88.140000,25.800000,38.959759,34.924965
42,US,87.817200,33.653808,39.783730,-100.445882
43,US-France,88.000000,50.000000,45.082085,6.059645
44,Ukraine,84.600000,13.000000,49.487197,31.271832
45,Uruguay,84.541176,25.847059,-32.875555,-56.020153


In [126]:
# Function to convert df to geoJson
def df_to_geojson(map_df, properties, lat='lat', lon='lon'):
    
    # create a new dict to contain geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}
    
    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in map_df.iterrows():
        
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Polygon',
                               'coordinates':[]}}
        
        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        
        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
            
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    return geojson

In [127]:
# Save list of columns in var
useful_columns = ['country', 'points', 'price']

# Save df in geojson dict with properties
geojson_dict = df_to_geojson(df_geo, properties=useful_columns)

# Finally dump json
geojson_str = json.dumps(geojson_dict, indent=2)

In [128]:
# save the geojson result to a file
output_filename = 'wine.js'
with open(output_filename, 'w') as output_file:
    output_file.write('var dataset = {};'.format(geojson_str))
    
# how many features did we save to the geojson file?
print('{} geotagged features saved to file'.format(len(geojson_dict['features'])))

46 geotagged features saved to file


In [104]:
# convert df to list of dictionaries for json later on
country_list = map_df.to_dict('records')

# Upolad map data to sqlite db

In [46]:
# Creating a Class & table 
class Map_wine(Base):
    __tablename__ = 'map_wine_table'
    id = Column(Integer, primary_key=True)
    country = Column(String(255))
    points = Column(Float)
    price = Column(Float)
    lat = Column(Float)
    lon = Column(Float)

In [47]:
Base.metadata.create_all(conn)

In [48]:
# Transfer the data to sqlite table
map_df.to_sql(name='map_wine_table', con=conn, if_exists='append', index=False)

In [49]:
# Inspecting the Table to make sure the table is there
inspector = inspect(engine)
inspector.get_table_names()

['map_wine_table', 'wine_table']

In [50]:
# Checking the data 
engine.execute('SELECT * FROM map_wine_table LIMIT 20').fetchall()

[(1, 'Albania', 88.0, 20.0, 41.000028, 19.9999619),
 (2, 'Argentina', 86.00107392160372, 20.794880973688922, -34.9964963, -64.9672817),
 (3, 'Australia', 87.88148753575807, 31.258479771148345, -24.7761086, 134.755),
 (4, 'Austria', 89.19089810712848, 31.192106322996377, 47.2000338, 13.199959),
 (5, 'Bosnia and Herzegovina', 84.75, 12.75, 44.3053476, 17.5961467),
 (6, 'Brazil', 83.24, 19.92, -10.3333333, -53.2),
 (7, 'Bulgaria', 85.46753246753246, 11.545454545454545, 42.6073975, 25.4856617),
 (8, 'Canada', 88.22164948453609, 34.628865979381445, 61.0666922, -107.9917071),
 (9, 'Chile', 86.29812695109261, 19.344779743322928, -31.7613365, -71.3187697),
 (10, 'China', 82.0, 20.333333333333332, 35.000074, 104.999927),
 (11, 'Croatia', 86.49397590361446, 23.10843373493976, 45.5643442, 17.0118954),
 (12, 'Cyprus', 85.87096774193549, 15.483870967741936, 34.9823018, 33.1451285),
 (13, 'Czech Republic', 85.83333333333333, 18.0, 49.8167003, 15.4749544),
 (14, 'England', 92.75, 47.5, 52.7954791, -0